# Model for the Toxic Comment Competition on Kaggle using FastText

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

Based on the kernel from Jeremy Howard published on kaggle: https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout

Instead of using the Glove embedding, this is using the fasttext embedding (crawl_30d-2M.vec). I preprocessed the embedding with embedding_utils.py in order to check for dimensions and remove the first line

Validiation Accuracy: 98,425% which is an improvement of about 0.2% compared to baseline model

In [1]:
import sys, os, re, csv, codecs, logging, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from sklearn.metrics import roc_auc_score

/anaconda3/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


We include the GloVe word vectors in our input files. To include these in your kernel, simple click 'input files' at the top of the notebook, and search 'glove' in the 'datasets' section.

In [2]:
path = 'data/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{path}crawl_300d-2M_processed.vec'
#EMBEDDING_FILE=f'{path}glove.6B.50d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv'
TEST_DATA_FILE=f'{path}{comp}test.csv'

Set some basic config parameters:

In [3]:
#embed_size = 50 # how big is each word vector
embed_size=300
max_features = 60000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 400 # max number of words in a comment to use

Read in our data and replace missing values:

In [4]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the embeddings vectors (space delimited strings) into a dictionary from word->vector.

In [6]:
def read_embedding_list(file_path):
    embedding_word_dict = {}
    embedding_list = []
    i = 0
    with open(file_path) as f:
        for row in f:
            data = row.split(" ")
            word = data[0]
            embedding = np.array([float(num) for num in data[1:-1]])
            embedding_list.append(embedding)
            embedding_word_dict[word] = embedding
 
    embedding_list = np.array(embedding_list)
    return embedding_list, embedding_word_dict

Use these vectors to create our embedding matrix, with random initialization for words that aren't in Fasttext. We'll use the same mean and stdev of embeddings the Fasttext has when generating the random init.

In [7]:
embedding_list, embeddings_index = read_embedding_list(EMBEDDING_FILE)

Determine mean values and standard deviation to initialize the embedding matrix. This ensures that all words which are not found in the embeddings are set to random value around the mean.

In [25]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.005528633668833318, 0.34703942181861597)

In [10]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Simple bidirectional LSTM with two fully connected layers. We add some dropout to the LSTM since even 2 epochs is enough to overfit.

In [11]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(inp)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we're ready to fit out model! Use `validation_split` when not submitting.

In [26]:
print(X_t.shape)
print(y.shape)

(159571, 400)
(159571, 6)


In [12]:
filepath="crawl_weights.best.model" + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', patience=2, mode='max') 
callbacks_list = [checkpoint, early_stop]

model.fit(X_t, y, batch_size=64, epochs=20, validation_split=0.1,callbacks=callbacks_list);

Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 2274s 16ms/step - loss: 0.0655 - acc: 0.9776 - val_loss: 0.0481 - val_acc: 0.9819

Epoch 00001: val_acc improved from -inf to 0.98187, saving model to crawl_weights.best.model.hdf5
Epoch 2/20
143613/143613 [==============================] - 2337s 16ms/step - loss: 0.0488 - acc: 0.9818 - val_loss: 0.0446 - val_acc: 0.9830

Epoch 00002: val_acc improved from 0.98187 to 0.98297, saving model to crawl_weights.best.model.hdf5
Epoch 3/20
143613/143613 [==============================] - 2248s 16ms/step - loss: 0.0456 - acc: 0.9825 - val_loss: 0.0432 - val_acc: 0.9836

Epoch 00003: val_acc improved from 0.98297 to 0.98363, saving model to crawl_weights.best.model.hdf5
Epoch 4/20
143613/143613 [==============================] - 2260s 16ms/step - loss: 0.0436 - acc: 0.9833 - val_loss: 0.0440 - val_acc: 0.9833

Epoch 00004: val_acc did not improve
Epoch 5/20
143613/143613 [==============

In [13]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv(f'{path}{comp}sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission.csv', index=False)

153164/153164 [==============================] - 933s 6ms/step


## Checking prediction with a sample from test set

In [17]:
test_sample = list_sentences_test[0:2].reshape(2,)
print(test_sample[0])
print("----------------------------------")
print(test_sample[1])

Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,
----------------------------------
== From RfC == 

 The title is fine as it is, IMO.


In [18]:
tokenized_sample = tokenizer.texts_to_sequences(test_sample)
X_sample = pad_sequences(tokenized_sample, maxlen=maxlen)

y_test = model.predict([X_sample],verbose=0)


In [20]:
print(y_test)

[[9.9999166e-01 3.9733967e-01 9.8811215e-01 3.5311554e-02 9.4863749e-01
  3.9724001e-01]
 [1.5717048e-04 3.3678338e-11 1.5220577e-06 4.5525203e-10 2.4704607e-07
  1.8052205e-08]]


In [21]:
heading = np.append(["comment"], [list_classes]).reshape(1,7)
dd = np.append(test_sample.reshape(2,1), y_test, axis=1)

index = ['Row'+str(i) for i in range(1, len(dd)+1)]
df = pd.DataFrame(dd, index, columns=heading[0,0:])
df.tail()


,comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
Row1,Yo bitch Ja Rule is more succesful then you'll...,0.999992,0.39734,0.988112,0.0353116,0.948637,0.39724
Row2,== From RfC == \n\n The title is fine as it is...,0.00015717,3.36783e-11,1.52206e-06,4.55252e-10,2.47046e-07,1.80522e-08


Row1 has prob. of being toxic, insulting and obscene

Row2 can be considered as being a clean comment